In [ ]:
import pandas as pd
import numpy as np

#Define your Excel file path
file_path = "..\\Traffic_updated.xlsx"  # Update with your actual file path

# Load the Excel file
xls = pd.ExcelFile(file_path)

# Read all sheets into a dictionary of DataFrames
dfs = {sheet_name: xls.parse(sheet_name) for sheet_name in xls.sheet_names}

# Display available sheets
print("Sheets in Excel file:", xls.sheet_names)


Sheets in Excel file: ['Vehicles', 'Drivers', 'Location', 'TrafficFlow', 'Accidents', 'RoadConditions', 'TrafficLights', 'Cameras', 'SpeedViolations', 'ParkingTickets', 'PublicTransport', 'TollTransactions', 'CongestionLevels', 'RoadClosures', 'EmergencyVehicles', 'WeatherData', 'PedestrianFlow', 'TrafficIncidents']


DATA EXTRACTION 

In [26]:
from sqlalchemy import create_engine

server = "DESKTOP-NU99729\SQLEXPRESS"  # Example: "DESKTOP-XXXX\SQLEXPRESS"
database = "Traffic"
#username = "your_username"
#password = "your_password"

# Create an engine for SQL Server
#engine = create_engine(f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server")
engine = create_engine(f"mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server")


<>:3: SyntaxWarning: invalid escape sequence '\S'
<>:3: SyntaxWarning: invalid escape sequence '\S'
C:\Users\hp\AppData\Local\Temp\ipykernel_10320\3362690236.py:3: SyntaxWarning: invalid escape sequence '\S'
  server = "DESKTOP-NU99729\SQLEXPRESS"  # Example: "DESKTOP-XXXX\SQLEXPRESS"


In [27]:
#engine = create_engine(f"mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server")

for sheet_name, df in dfs.items():
    df.to_sql(sheet_name, engine, if_exists="replace", index=False)
    print(f"Loaded {sheet_name} into SQL Server")


Loaded Vehicles into SQL Server
Loaded Drivers into SQL Server
Loaded Location into SQL Server
Loaded TrafficFlow into SQL Server
Loaded Accidents into SQL Server
Loaded RoadConditions into SQL Server
Loaded TrafficLights into SQL Server
Loaded Cameras into SQL Server
Loaded SpeedViolations into SQL Server
Loaded ParkingTickets into SQL Server
Loaded PublicTransport into SQL Server
Loaded TollTransactions into SQL Server
Loaded CongestionLevels into SQL Server
Loaded RoadClosures into SQL Server
Loaded EmergencyVehicles into SQL Server
Loaded WeatherData into SQL Server
Loaded PedestrianFlow into SQL Server
Loaded TrafficIncidents into SQL Server


In [ ]:
#SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE';


In [102]:
# Fetch all table names
query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'"
table_names = pd.read_sql(query, engine)

# Load all tables into a dictionary of DataFrames
dfs = {table: pd.read_sql(f"SELECT * FROM {table}", engine) for table in table_names["TABLE_NAME"]}

# Display loaded table names
print("Loaded tables:", list(dfs.keys()))

Loaded tables: ['sysdiagrams', 'spt_fallback_db', 'spt_fallback_dev', 'spt_fallback_usg', 'spt_monitor', 'MSreplication_options', 'Vehicles', 'Drivers', 'Location', 'TrafficFlow', 'Accidents', 'RoadConditions', 'TrafficLights', 'Cameras', 'SpeedViolations', 'ParkingTickets', 'PublicTransport', 'TollTransactions', 'CongestionLevels', 'RoadClosures', 'EmergencyVehicles', 'WeatherData', 'PedestrianFlow', 'TrafficIncidents']


In [103]:
# Display first few rows of each table
for table, df in dfs.items():
    print(f"Table: {table}")
    print(df.head(), "\n")  # Show first 5 rows


Table: sysdiagrams
        name  principal_id  diagram_id  version  \
0  Diagram_0             1           1        1   

                                          definition  
0  b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...   

Table: spt_fallback_db
Empty DataFrame
Columns: [xserver_name, xdttm_ins, xdttm_last_ins_upd, xfallback_dbid, name, dbid, status, version]
Index: [] 

Table: spt_fallback_dev
Empty DataFrame
Columns: [xserver_name, xdttm_ins, xdttm_last_ins_upd, xfallback_low, xfallback_drive, low, high, status, name, phyname]
Index: [] 

Table: spt_fallback_usg
Empty DataFrame
Columns: [xserver_name, xdttm_ins, xdttm_last_ins_upd, xfallback_vstart, dbid, segmap, lstart, sizepg, vstart]
Index: [] 

Table: spt_monitor
                  lastrun  cpu_busy  io_busy  idle  pack_received  pack_sent  \
0 2016-04-30 00:46:37.563        11        6   703             39         39   

   connections  pack_errors  total_read  total_write  total_errors  
0           34            0   

In [104]:
# Unpack all tables dynamically into separate DataFrames
for table_name, df in dfs.items():
    globals()[table_name] = df  # Create a variable with the table name

# Verify that the tables are now separate DataFrames
print("DataFrames created:", list(dfs.keys()))

# Tables to drop
tables_to_drop = [
    'ParkingTickets', 
    'PublicTransport', 
    'TollTransactions', 
    'CongestionLevels', 
    'RoadClosures', 
    'EmergencyVehicles',
    'PedestrianFlow'
]

# Create a new dictionary excluding the unwanted tables i.e. Remove keys from the original dictionary
for table in tables_to_drop:
    if table in dfs:
        del dfs[table]

print("Remaining Tables:", list(dfs.keys()))

DataFrames created: ['sysdiagrams', 'spt_fallback_db', 'spt_fallback_dev', 'spt_fallback_usg', 'spt_monitor', 'MSreplication_options', 'Vehicles', 'Drivers', 'Location', 'TrafficFlow', 'Accidents', 'RoadConditions', 'TrafficLights', 'Cameras', 'SpeedViolations', 'ParkingTickets', 'PublicTransport', 'TollTransactions', 'CongestionLevels', 'RoadClosures', 'EmergencyVehicles', 'WeatherData', 'PedestrianFlow', 'TrafficIncidents']
Remaining Tables: ['sysdiagrams', 'spt_fallback_db', 'spt_fallback_dev', 'spt_fallback_usg', 'spt_monitor', 'MSreplication_options', 'Vehicles', 'Drivers', 'Location', 'TrafficFlow', 'Accidents', 'RoadConditions', 'TrafficLights', 'Cameras', 'SpeedViolations', 'WeatherData', 'TrafficIncidents']


In [105]:
# Table: Expected Data Types
expected_data_types = {
    "Vehicles": {
        "VehicleID": np.int64,
        "PlateNumber": object,
        "VehicleType": object,
        "OwnerID": np.int64,
    },
    "Drivers": {
        "DriverID": np.int64,
        "Name": object,
        "Age": np.int64,
        "LicenseNumber": object,
        "ExperienceYears": np.int64,
    },
    "TrafficFlow": {
        "FlowID": np.int64,
        "Location": object,
        "VehicleCount": np.int64,
        "Timestamp": 'datetime',
    },
    "Location": {
        "Location ID": np.int64,
        "Road Name": object,
        "Area/Location": object,
    },
    "Accidents": {
        "AccidentID": np.int64,
        "Location": object,
        "Severity": object,
        "VehiclesInvolved": np.int64,
        "ReportedAt": 'datetime',
    },
    "Road Conditions": {
        "ConditionID": np.int64,
        "Location": object,
        "Surface": object,
        "Visibility": object,
        "RecordedAt": 'datetime',
    },
    "Traffic Lights": {  # Add Traffic Lights
        "LightID": np.int64,
        "Location": object,
        "Status": object,
        "LastChanged": 'datetime',
    },
    "Cameras": {  # Add Cameras
        "CameraID": np.int64,
        "Location": object,
        "Status": object,
        "LastMaintenance": 'datetime',
    },
    "Speed Violations": {
        "ViolationID": np.int64,
        "VehicleID": np.int64,
        "SpeedRecorded": np.float64,
        "SpeedLimit": np.float64,
        "Timestamp": 'datetime',
    },
    "Weather Data": {
        "WeatherID": np.int64,
        "Temperature_C": np.float64,
        "Humidity_Percent": np.int64,
        "Condition": object,
        "Timestamp": 'datetime',
    },
    "Traffic Incidents": {
        "IncidentID": np.int64,
        "Type": object,
        "Location": object,
        "ReportedAt": 'datetime',
    },
}

# Function 1: Validate Data Types
def validate_data_types(df, table_name):
    """
    Validates data types against expected types in a DataFrame, converting to datetime where necessary.
    
    Parameters:
        df (pd.DataFrame): The DataFrame to validate.
        table_name (str): The name of the table being validated.
    
    Returns:
        dict: Dictionary of validation results for each column.
    """
    results = {}
    expected_types = expected_data_types.get(table_name, {})
    
    for column, expected_type in expected_types.items():
        if column in df.columns:
            actual_type = df[column].dtype
            
            # Special handling for datetime: convert and validate
            if expected_type == 'datetime':
                try:
                    # Attempt conversion
                    pd.to_datetime(df[column], errors='raise')
                    results[column] = True  # Datetime conversion successful
                except ValueError:
                    results[column] = False  # Datetime conversion failed
            
            # Generic handling for other types
            elif expected_type == object and pd.api.types.is_string_dtype(actual_type):
                results[column] = True
            elif expected_type == np.int64 and pd.api.types.is_integer_dtype(actual_type):
                results[column] = True
            elif expected_type == np.float64 and pd.api.types.is_float_dtype(actual_type):
                results[column] = True
            else:
                results[column] = False
        else:
            results[column] = False
    
    return results

def validate_missing_values(df):
    results = {}
    for column in df.columns:
        results[column] = df[column].isnull().sum()
    return results

def validate_integer_outliers(df):
    results = {}
    for column in df.columns:
        if pd.api.types.is_integer_dtype(df[column]):
            Q1 = df[column].quantile(0.25)
            Q3 = df[column].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)][column].count()
            results[column] = outliers
        else:
            results[column] = 0
    return results

# Function to validate data for each dataframe
def validate_data_for_table(df, table_name):
    print(f"Validating table: {table_name}")
    
    # Validate data types
    data_type_results = validate_data_types(df, table_name)
    print(f"Data type validation results: {data_type_results}")
    
    # Validate missing values
    missing_values_results = validate_missing_values(df)
    print(f"Missing values results: {missing_values_results}")

    # Validate integer outliers
    outliers_results = validate_integer_outliers(df)
    print(f"Outlier validation results: {outliers_results}")

# Apply validation for each relevant DataFrame
validate_data_for_table(dfs["Vehicles"], "Vehicles")
validate_data_for_table(dfs["Drivers"], "Drivers")
validate_data_for_table(dfs["TrafficFlow"], "TrafficFlow")
validate_data_for_table(dfs["Location"], "Location")
validate_data_for_table(dfs["Accidents"], "Accidents")
validate_data_for_table(dfs["RoadConditions"], "Road Conditions")
validate_data_for_table(dfs["TrafficLights"], "Traffic Lights")  # Validate Traffic Lights
validate_data_for_table(dfs["Cameras"], "Cameras")  # Validate Cameras
validate_data_for_table(dfs["SpeedViolations"], "Speed Violations")
validate_data_for_table(dfs["WeatherData"], "Weather Data")
validate_data_for_table(dfs["TrafficIncidents"], "Traffic Incidents")

Validating table: Vehicles
Data type validation results: {'VehicleID': True, 'PlateNumber': True, 'VehicleType': True, 'OwnerID': True}
Missing values results: {'VehicleID': np.int64(0), 'PlateNumber': np.int64(0), 'VehicleType': np.int64(0), 'OwnerID': np.int64(0)}
Outlier validation results: {'VehicleID': np.int64(0), 'PlateNumber': 0, 'VehicleType': 0, 'OwnerID': np.int64(0)}
Validating table: Drivers
Data type validation results: {'DriverID': True, 'Name': True, 'Age': True, 'LicenseNumber': True, 'ExperienceYears': True}
Missing values results: {'DriverID': np.int64(0), 'Name': np.int64(0), 'Age': np.int64(0), 'LicenseNumber': np.int64(0), 'ExperienceYears': np.int64(0)}
Outlier validation results: {'DriverID': np.int64(0), 'Name': 0, 'Age': np.int64(0), 'LicenseNumber': 0, 'ExperienceYears': np.int64(0)}
Validating table: TrafficFlow
Data type validation results: {'FlowID': True, 'Location': False, 'VehicleCount': True, 'Timestamp': True}
Missing values results: {'FlowID': np.int

In [106]:
 #Validate Data Types for Each Table
for table_name, df in dfs.items():
    print(f"Validating {table_name} Table:")
    print("Data Type Validation:", validate_data_types(df, table_name))
    print("Missing Values Validation:", validate_missing_values(df))
    print("Integer Outliers Validation:", validate_integer_outliers(df))
    print("\n")

Validating sysdiagrams Table:
Data Type Validation: {}
Missing Values Validation: {'name': np.int64(0), 'principal_id': np.int64(0), 'diagram_id': np.int64(0), 'version': np.int64(0), 'definition': np.int64(0)}
Integer Outliers Validation: {'name': 0, 'principal_id': np.int64(0), 'diagram_id': np.int64(0), 'version': np.int64(0), 'definition': 0}


Validating spt_fallback_db Table:
Data Type Validation: {}
Missing Values Validation: {'xserver_name': np.int64(0), 'xdttm_ins': np.int64(0), 'xdttm_last_ins_upd': np.int64(0), 'xfallback_dbid': np.int64(0), 'name': np.int64(0), 'dbid': np.int64(0), 'status': np.int64(0), 'version': np.int64(0)}
Integer Outliers Validation: {'xserver_name': 0, 'xdttm_ins': 0, 'xdttm_last_ins_upd': 0, 'xfallback_dbid': 0, 'name': 0, 'dbid': 0, 'status': 0, 'version': 0}


Validating spt_fallback_dev Table:
Data Type Validation: {}
Missing Values Validation: {'xserver_name': np.int64(0), 'xdttm_ins': np.int64(0), 'xdttm_last_ins_upd': np.int64(0), 'xfallback_l

TRANSFORMATION 

In [107]:
# Helper Functions
def clean_text(text):
    """
    Removes leading/trailing whitespace and converts text to title case.
    """
    if isinstance(text, str):
        return text.strip()
    return text

def calculate_experience(license_issue_date):
    """
    Calculates years of driving experience.
    """
    if pd.isnull(license_issue_date):
        return None
    today = pd.to_datetime('today')
    return (today - pd.to_datetime(license_issue_date)).days / 365.25

def convert_to_boolean(value):
    """
    Converts text to boolean.
    """
    if isinstance(value, str):
        value = value.strip().lower()
        if value == 'y':
            return True
        elif value == 'n':
            return False
    return None

def time_of_day(time):
    """
    Classifies the time of day.
    """
    if isinstance(time, pd.Timestamp):
        hour = time.hour
        if 6 <= hour < 12:
            return "Morning"
        elif 12 <= hour < 18:
            return "Afternoon"
        elif 18 <= hour < 24:
            return "Evening"
        else:
            return "Night"
    return None

def month_name(date):
    """
    Extracts month name from a date.
    """
    if isinstance(date, pd.Timestamp):
        return date.strftime("%B")
    return None

# Transformations for Star Schema
transformations = {
    "Vehicles": {
        "PlateNumber": lambda x: clean_text(x),
    },
    "Drivers": {
        "ExperienceYears": lambda x: float(x) if pd.notnull(x) else None,  # Standardize to float
    },
    "Speed Violations": {
        "Timestamp": lambda x: pd.to_datetime(x),  # Standardize timestamp format
    },
    "Accidents": {
        "ReportedAt": lambda x: pd.to_datetime(x),  # Standardize timestamp format
    },
    "Weather Data": {
        "Timestamp": lambda x: pd.to_datetime(x),  # Standardize timestamp format
        "Humidity_Percent": lambda x: float(x) / 100 if pd.notnull(x) else None,  # Convert to decimal
    },
    "Traffic Incidents": {
        "ReportedAt": lambda x: pd.to_datetime(x),  # Standardize timestamp format
    },
    "Location": {
        "Road Name": lambda x: clean_text(x),
        "Area/Location": lambda x: clean_text(x),
    },
    "TrafficFlow": {
        "Timestamp": lambda x: pd.to_datetime(x),  # Standardize timestamp format
    },
    "Traffic Lights": {
        "LastChanged": lambda x: pd.to_datetime(x),  # Standardize date format
    },
    "Cameras": {
        "LastMaintenance": lambda x: pd.to_datetime(x),  # Standardize date format
    },
}


In [127]:

# Transforming DataFrames
for table_name, df in dfs.items():
    if table_name in transformations:
        for col, transform in transformations[table_name].items():
            if col in df.columns:
                df[col] = df[col].apply(transform)

    # Create 'has_violated_speed' variable in Vehicles DataFrame
    if table_name == 'Vehicles' and 'VehicleID' in df.columns:
        vehicle_ids_in_speed_violations = set(dfs['SpeedViolations']['VehicleID'].unique())
        df['has_violated_speed'] = df['VehicleID'].apply(lambda x: x in vehicle_ids_in_speed_violations)

    # Create new variables for Accidents DataFrames
    if table_name == 'Accidents':
        df['acc_date'] = pd.to_datetime(df['ReportedAt']).dt.date
        df['acc_time'] = pd.to_datetime(df['ReportedAt']).dt.time
        df['acc_day'] = pd.to_datetime(df['ReportedAt']).dt.strftime('%A')
        df['acc_day_month'] = pd.to_datetime(df['ReportedAt']).dt.strftime('%d/%m')

   # Create new variables for Speed Violations DataFrames
    if table_name == 'SpeedViolations':
        df['violation_date'] = pd.to_datetime(df['Timestamp']).dt.date
        df['violation_time'] = pd.to_datetime(df['Timestamp']).dt.time
     # Create new variables for Traffic Flow DataFrames
    
    if table_name == 'TrafficFlow':
        df['traffic_flow_date'] = pd.to_datetime(df['Timestamp']).dt.date
        df['traffic_flow_time'] = pd.to_datetime(df['Timestamp']).dt.time

    # Create new variables for Traffic Incidents DataFrames
    if table_name == 'TrafficIncidents':
        df['inc_date'] = pd.to_datetime(df['ReportedAt']).dt.date
        df['inc_time'] = pd.to_datetime(df['ReportedAt']).dt.time
        df['inc_day'] = pd.to_datetime(df['ReportedAt']).dt.strftime('%A')
        df['inc_day_month'] = pd.to_datetime(df['ReportedAt']).dt.strftime('%d/%m')

# Print Transformed DataFrames
for table_name, df in dfs.items():
    print(f"\nTransformed {table_name} DataFrame:")
    print(df.head())


Transformed sysdiagrams DataFrame:
        name  principal_id  diagram_id  version  \
0  Diagram_0             1           1        1   

                                          definition  
0  b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...  

Transformed spt_fallback_db DataFrame:
Empty DataFrame
Columns: [xserver_name, xdttm_ins, xdttm_last_ins_upd, xfallback_dbid, name, dbid, status, version]
Index: []

Transformed spt_fallback_dev DataFrame:
Empty DataFrame
Columns: [xserver_name, xdttm_ins, xdttm_last_ins_upd, xfallback_low, xfallback_drive, low, high, status, name, phyname]
Index: []

Transformed spt_fallback_usg DataFrame:
Empty DataFrame
Columns: [xserver_name, xdttm_ins, xdttm_last_ins_upd, xfallback_vstart, dbid, segmap, lstart, sizepg, vstart]
Index: []

Transformed spt_monitor DataFrame:
                  lastrun  cpu_busy  io_busy  idle  pack_received  pack_sent  \
0 2016-04-30 00:46:37.563        11        6   703             39         39   

   connections  pack_e

In [124]:
dfs["TrafficFlow"]

,FlowID,Location ID,Location,Area/Location,VehicleCount,Timestamp,traffic_flow_date,traffic_flow_time
0,1,1,Shahrah-e-Faisal,Jinnah International Airport to PIDC,418,2025-02-15 22:08:31,2025-02-15,22:08:31
1,2,2,MA Jinnah Road,Saddar to Kharadar,992,2025-02-05 18:12:09,2025-02-05,18:12:09
2,3,3,II Chundrigar Road,Tower to Merewether Tower,793,2025-02-17 07:56:45,2025-02-17,07:56:45
3,4,4,University Road,NED University to Hassan Square,804,2025-02-28 05:24:57,2025-02-28,05:24:57
4,5,5,Rashid Minhas Road,Gulshan to Gulistan-e-Jauhar,190,2025-02-26 03:51:13,2025-02-26,03:51:13
...,...,...,...,...,...,...,...,...
145,146,89,Shahrah-e-Gulshan Chowrangi,Gulshan-e-Iqbal,740,2025-02-06 07:58:52,2025-02-06,07:58:52
146,147,90,Shahrah-e-North Karachi,North Karachi,167,2025-02-10 02:33:16,2025-02-10,02:33:16
147,148,65,Shahrah-e-PECHS,PECHS,619,2025-02-06 22:57:51,2025-02-06,22:57:51
148,149,68,Shahrah-e-Jauhar,Gulistan-e-Jauhar,911,2025-02-21 06:01:55,2025-02-21,06:01:55


In [128]:
dfs["TrafficIncidents"]

,IncidentID,Type,Location,ReportedAt,inc_date,inc_time,inc_day,inc_day_month
0,1,Vehicle Breakdown,Shahrah-e-Faisal,2025-02-15 14:46:00,2025-02-15,14:46:00,Saturday,15/02
1,2,Vehicle Breakdown,MA Jinnah Road,2025-02-23 18:29:41,2025-02-23,18:29:41,Sunday,23/02
2,3,Road Rage,II Chundrigar Road,2025-01-15 23:55:50,2025-01-15,23:55:50,Wednesday,15/01
3,4,Vehicle Breakdown,University Road,2025-02-05 22:04:32,2025-02-05,22:04:32,Wednesday,05/02
4,5,Road Rage,Rashid Minhas Road,2025-01-15 19:18:05,2025-01-15,19:18:05,Wednesday,15/01
...,...,...,...,...,...,...,...,...
95,96,Road Rage,Shahrah-e-Sindhi Muslim Society,2025-01-27 07:13:52,2025-01-27,07:13:52,Monday,27/01
96,97,Vehicle Breakdown,Shahrah-e-Gulistan-e-Johar,2025-02-20 19:36:26,2025-02-20,19:36:26,Thursday,20/02
97,98,Road Rage,Shahrah-e-North Nazimabad,2025-02-10 07:25:01,2025-02-10,07:25:01,Monday,10/02
98,99,Vehicle Breakdown,Shahrah-e-Farooq-e-Azam,2025-01-11 21:53:35,2025-01-11,21:53:35,Saturday,11/01


In [120]:
dfs["Vehicles"]

,VehicleID,PlateNumber,VehicleType,OwnerID,has_violated_speed
0,1,92X 5338,Car,64,True
1,2,186-ZJJ,Car,145,False
2,3,835 1WK,Car,140,True
3,4,OYD7711,Bus,70,False
4,5,KFU-0955,Bus,24,False
...,...,...,...,...,...
195,196,9MR 974,Car,112,False
196,197,AI-9234,Bus,60,True
197,198,6538,Motorcycle,72,False
198,199,XPA6114,Motorcycle,128,False


In [143]:
#DIM VEHICLE
dim_vehicle = dfs['Vehicles'][[
    'VehicleID',
    'PlateNumber',
    'VehicleType',
    'OwnerID',
    'has_violated_speed'
]].rename(columns={'VehicleID': 'vehicle_id', 'PlateNumber': 'plate_number', 'VehicleType': 'vehicle_type', 'OwnerID': 'driver_id'})
dim_vehicle = dim_vehicle.drop_duplicates(subset=['vehicle_id'])

# 2. DIM SPEED VIOLATION
dim_speed_violation = dfs['SpeedViolations'][[
    'ViolationID',
    'VehicleID',
    'SpeedLimit',
    'violation_date',
    'violation_time'
]].rename(columns={'ViolationID': 'violation_id', 'VehicleID': 'vehicle_id', 'SpeedLimit': 'speed_limit'})
dim_speed_violation = dim_speed_violation.drop_duplicates(subset=['violation_id'])

# 3. DIM DRIVER
dim_driver = dfs['Drivers'][[
    'DriverID',
    'Name',
    'ExperienceYears',
    'LicenseNumber'
]].rename(columns={'DriverID': 'driver_id', 'Name': 'driver_name', 'ExperienceYears': 'driver_experience', 'LicenseNumber': 'license_number'})
dim_driver = dim_driver.drop_duplicates(subset=['driver_id'])

# 4. DIM INCIDENT
dim_incident = dfs['TrafficIncidents'][[
    'IncidentID',
    'Location',
    'inc_date',
    'inc_time',
    'inc_day'
]].rename(columns={'IncidentID': 'incident_id', 'Location': 'location_id', 'inc_date': 'incident_date', 'inc_time': 'incident_time', 'inc_day': 'acc_day'})
dim_incident = dim_incident.drop_duplicates(subset=['incident_id'])

# 5. DIM LOCATION
# 5. DIM LOCATION
dim_location = dfs['Location'][[
    'Location ID',
    'Road Name',
    'Area/Location'
]].rename(columns={'Location ID': 'location_id', 'Road Name': 'location_name', 'Area/Location': 'location_Area'})

# Merge traffic light information
dim_location = pd.merge(dim_location, dfs['TrafficLights'][['Location', 'Status', 'LastChanged']],
                        left_on='location_name', right_on='Location', how='left')
dim_location = dim_location.rename(columns={'Status': 'traffic_light_status', 'LastChanged': 'traffic_light_maintenance_date'})
dim_location.drop('Location', axis=1, inplace=True)

# Merge camera information
dim_location = pd.merge(dim_location, dfs['Cameras'][['Location', 'Status', 'LastMaintenance']],
                        left_on='location_name', right_on='Location', how='left')
dim_location = dim_location.rename(columns={'Status': 'camera_status', 'LastMaintenance': 'camera_last_maintenance_date'})
dim_location.drop('Location', axis=1, inplace=True)

dim_location = dim_location.drop_duplicates(subset=['location_id'])

# 6. DIM ACCIDENT
dim_accident = dfs['Accidents'][[
    'AccidentID',
    'Location',
    'acc_date',
    'acc_time'
]].rename(columns={'AccidentID': 'accident_id', 'Location': 'location_id', 'acc_date': 'accident_date', 'acc_time': 'accident_time'})

# Merge road condition information
dim_accident = pd.merge(dim_accident, dfs['RoadConditions'][['Location', 'ConditionID', 'Surface']],
                        left_on='location_id', right_on='Location', how='left')

dim_accident = dim_accident.rename(columns={'ConditionID': 'road_condition_id', 'Surface': 'road_surface'})
dim_accident.drop('Location', axis=1, inplace=True)
dim_accident = dim_accident.drop_duplicates(subset=['accident_id'])

In [145]:

# 7. DIM DATE
dim_date = pd.DataFrame()
# Use unique dates from both Accidents and Traffic Incidents
accident_dates = pd.to_datetime(dfs['Accidents']['acc_date']).dt.date
incident_dates = pd.to_datetime(dfs['TrafficIncidents']['inc_date']).dt.date
all_dates = pd.concat([accident_dates, incident_dates]).unique()

dim_date['date_id'] = all_dates  # Store unique dates here
dim_date['accident_date'] = pd.to_datetime(dim_date['date_id']).dt.date
dim_date['accident_month'] = pd.to_datetime(dim_date['accident_date']).dt.month_name()
dim_date['day_of_week'] = pd.to_datetime(dim_date['accident_date']).dt.day_name()
dim_date['accident_time'] = pd.to_datetime(dim_date['accident_date']).dt.time
dim_date['accident_time_of_day'] = pd.to_datetime(dim_date['accident_date']).apply(time_of_day)

dim_date = dim_date.drop_duplicates(subset=['date_id'])
# 8. DIM WEATHER DATA
dim_weather_data = dfs['WeatherData'][[
    'WeatherID',
    'Timestamp',
    'Temperature_C',
    'Humidity_Percent',
    'Condition'
]].rename(columns={'WeatherID': 'weather_id', 'Timestamp': 'weather_date', 'Temperature_C': 'weather_temperature', 'Humidity_Percent': 'weather_humidity', 'Condition': 'weather_condition'})
dim_weather_data = dim_weather_data.drop_duplicates(subset=['weather_id'])

# 9. DIM TRAFFIC FLOW
dim_traffic_flow = dfs['TrafficFlow'][[
    'FlowID',
    'Location ID',
    'VehicleCount',
    'traffic_flow_date',
    'traffic_flow_time',
]].rename(columns={'FlowID': 'flow_id', 'Location ID': 'location_id', 'VehicleCount': 'vehicle_count'})
dim_traffic_flow = dim_traffic_flow.drop_duplicates(subset=['flow_id'])


In [139]:

# 11. Output the dimension and fact tables
print("\nDIM VEHICLE:")
print(dim_vehicle.head())

print("\nDIM SPEED VIOLATION:")
print(dim_speed_violation.head())

print("\nDIM DRIVER:")
print(dim_driver.head())

print("\nDIM INCIDENT:")
print(dim_incident.head())

print("\nDIM LOCATION:")
print(dim_location.head())

print("\nDIM ACCIDENT:")
print(dim_accident.head())

print("\nDIM DATE:")
print(dim_date.head())

print("\nDIM WEATHER DATA:")
print(dim_weather_data.head())

print("\nDIM TRAFFIC FLOW:")
print(dim_traffic_flow.head())



DIM VEHICLE:
   vehicle_id plate_number vehicle_type  driver_id  has_violated_speed
0           1     92X 5338          Car         64                True
1           2      186-ZJJ          Car        145               False
2           3      835 1WK          Car        140                True
3           4      OYD7711          Bus         70               False
4           5     KFU-0955          Bus         24               False

DIM SPEED VIOLATION:
   violation_id  vehicle_id  speed_limit violation_date violation_time
0             1         179           80     2025-02-12       16:54:07
1             2          41           80     2025-02-23       02:07:01
2             3         169           70     2025-03-03       09:24:10
3             4           6           70     2025-02-03       03:45:09
4             5         106           70     2025-02-14       06:10:16

DIM DRIVER:
   driver_id      driver_name  driver_experience license_number
0          1   Muhammad Ahmed      

In [141]:
# 10. FACT TRAFFIC EVENT
fact_traffic_event = pd.DataFrame()
fact_traffic_event['event_id'] = range(1, len(dfs['Accidents']) + len(dfs['TrafficIncidents']) + 1)
fact_traffic_event['vehicle_id'] =  dim_vehicle.vehicle_id
fact_traffic_event['driver_id'] =   dim_driver.driver_id
fact_traffic_event['location_id'] =  dim_location.location_id
fact_traffic_event['incident_date'] =dim_incident.incident_date
fact_traffic_event['accident_date'] =dim_accident.accident_date
fact_traffic_event['accident_time (hrs)'] =  dim_accident.accident_time
fact_traffic_event['incident_time (hrs)'] =dim_incident.incident_time
fact_traffic_event['accident_severity'] =  Accidents.Severity
fact_traffic_event['acc_vehicles_involved'] =Accidents.VehiclesInvolved
fact_traffic_event['road_visibility'] =RoadConditions.Visibility
fact_traffic_event['driver_age'] =Drivers.Age
fact_traffic_event['driver_experience'] = Drivers.ExperienceYears
fact_traffic_event['speed_recorded'] = SpeedViolations.SpeedRecorded
fact_traffic_event['weather_condition'] = dim_weather_data["weather_condition"] 
print("\nFACT TRAFFIC EVENT:")

print(fact_traffic_event.head(10))


FACT TRAFFIC EVENT:
   event_id  vehicle_id  driver_id  location_id incident_date accident_date  \
0         1           1          1          1.0    2025-02-15    2025-02-20   
1         2           2          2          2.0    2025-02-23    2025-01-07   
2         3           3          3          3.0    2025-01-15    2025-02-03   
3         4           4          4          4.0    2025-02-05    2025-02-19   
4         5           5          5          5.0    2025-01-15    2025-02-15   
5         6           6          6          6.0    2025-01-20    2025-01-06   
6         7           7          7          7.0    2025-01-30    2025-02-01   
7         8           8          8          8.0    2025-01-31    2025-02-01   
8         9           9          9          9.0    2025-03-03    2025-01-13   
9        10          10         10         10.0    2025-02-17    2025-02-24   

  accident_time (hrs) incident_time (hrs) accident_severity  \
0            15:15:34            14:46:00     